In [59]:
import numpy as np
import pandas as pd
from collections import Counter
import math

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score
from sklearn import metrics
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.grid_search import GridSearchCV
from sklearn import svm
from sklearn.decomposition import RandomizedPCA
from sklearn.metrics import classification_report
from sklearn import preprocessing
import matplotlib.pyplot as plt
import scipy.optimize as opt
import glob
import os
import sys
import pickle
import time
import timeit

from sklearn.decomposition import RandomizedPCA

import theano
import theano.tensor as T
import lasagne

#np.random.seed(1337)  # for reproducibility
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten, Layer, Reshape, AutoEncoder, Merge
from keras.layers.convolutional import Convolution2D,MaxPooling2D,Convolution1D,MaxPooling1D,UpSampling2D, ZeroPadding2D
from keras.optimizers import SGD, Adam, RMSprop, Adadelta
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
import keras.utils.visualize_util as vutil
from keras.layers.recurrent import LSTM, GRU, SimpleRNN
from keras.regularizers import l1, l2, l1l2, activity_l2, activity_l1
import keras.layers.containers as containers
from keras.layers.noise import GaussianNoise
from keras.regularizers import ActivityRegularizer
from keras import backend as K
import keras.utils.visualize_util as vutil
from keras.models import model_from_json

import seaborn as sns

%matplotlib inline

from IPython.display import SVG

import h5py

np.set_printoptions(suppress=True)  #Supress exponent of the number 2e-2 == 0.02

%matplotlib inline

In [60]:
#PCSI0048

In [61]:
#Read the data

GenomicFeatures_PCSI0048 = pd.read_csv(open('data/PCSI0048/GenomicFeatures_PCSI0048.txt','r'),sep='\t')
SNVCalls_PCSI0048    = pd.read_csv(open('data/PCSI0048/SNVCalls_PCSI0048.txt','r'),sep='\t') 

In [62]:
print GenomicFeatures_PCSI0048.shape
print SNVCalls_PCSI0048.shape

(125625, 15)
(125625, 137)


In [63]:
np.sum(GenomicFeatures_PCSI0048.isnull())

CHROM                  0
POS                    0
RefAllele              0
NonRefAllele           0
BaseQual               0
TumourCoverage         0
NormalCoverage         0
MapQual                0
ReadPosition          77
Trinucleotide          0
HomopolymerRate        0
GCcontent              0
GenomicLocation    33973
DistanceSNP            0
StrandBias            77
dtype: int64

In [64]:
GenomicFeatures_PCSI0048['ReadPosition'].fillna(0, inplace=True)  #Replace na with 0
GenomicFeatures_PCSI0048['StrandBias'].fillna(0, inplace=True)

#print np.sum(GenomicFeatures_PCSI0023.isnull())

In [65]:
np.unique(GenomicFeatures_PCSI0048['GenomicLocation'])

array(['UTR3', 'UTR5', 'downstream', 'exonic', 'exonic;splicing',
       'intergenic', 'intronic', 'nan', 'ncRNA_UTR3', 'ncRNA_UTR5',
       'ncRNA_exonic', 'ncRNA_intronic', 'ncRNA_splicing', 'splicing',
       'upstream', 'upstream;downstream'], 
      dtype='|S19')

In [66]:
GenomicFeatures_PCSI0048.head()

,CHROM,POS,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias
0,chr1,10583,47,9,1519,64,69,29,74,GCA,2.026144,0.716418,NaN,50859,0.281250
1,chr1,14111,117,9,3971,139,119,0,48,CTT,2.748201,0.512438,downstream,47331,0.482014
2,chr1,16206,124,21,4497,155,145,16,44,CTG,2.101911,0.597015,ncRNA_intronic,45236,0.812903
3,chr1,16993,122,16,4151,144,151,0,44,CCT,2.406897,0.592040,ncRNA_exonic,44449,0.736111
4,chr1,17594,116,26,4419,146,178,0,50,CCT,2.605634,0.631841,ncRNA_intronic,43848,0.472603


In [67]:
SNVCalls_PCSI0048.head()

,CHROM,POS,END,X2399010,X2642563,X2642565,X2673220,X2673230,X2701045,X2701049,X2811106,X2811111,X2811113,X2811116,X2811397,X2812914,X2813253,X2813265,X2813385,X2813460,
0,chr1,10582,10583,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...
1,chr1,14110,14111,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...
2,chr1,16205,16206,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...
3,chr1,16992,16993,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...
4,chr1,17593,17594,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...


In [68]:
np.unique(GenomicFeatures_PCSI0048['Trinucleotide'])

array(['ACA', 'ACC', 'ACG', 'ACN', 'ACT', 'ATA', 'ATC', 'ATG', 'ATT',
       'CCA', 'CCC', 'CCG', 'CCT', 'CTA', 'CTC', 'CTG', 'CTT', 'GCA',
       'GCC', 'GCG', 'GCT', 'GTA', 'GTC', 'GTG', 'GTT', 'TCA', 'TCC',
       'TCG', 'TCN', 'TCT', 'TTA', 'TTC', 'TTG', 'TTT'], 
      dtype='|S3')

In [69]:
PCSI0048 = pd.merge(GenomicFeatures_PCSI0048, SNVCalls_PCSI0048,left_on=['CHROM', 'POS'], right_on=['CHROM','END'])


In [70]:
PCSI0048.head() #182+15

,CHROM,POS_x,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias,POS_y,END,X2399010,X2642563,X2642565,
0,chr1,10583,47,9,1519,64,69,29,74,GCA,2.026144,0.716418,NaN,50859,0.281250,10582,10583,0,0,0,...
1,chr1,14111,117,9,3971,139,119,0,48,CTT,2.748201,0.512438,downstream,47331,0.482014,14110,14111,0,0,0,...
2,chr1,16206,124,21,4497,155,145,16,44,CTG,2.101911,0.597015,ncRNA_intronic,45236,0.812903,16205,16206,0,0,0,...
3,chr1,16993,122,16,4151,144,151,0,44,CCT,2.406897,0.592040,ncRNA_exonic,44449,0.736111,16992,16993,0,0,0,...
4,chr1,17594,116,26,4419,146,178,0,50,CCT,2.605634,0.631841,ncRNA_intronic,43848,0.472603,17593,17594,0,0,0,...


In [71]:
PCSI0048_X = PCSI0048.iloc[:,2:15] #Exclude CHROM, POS
PCSI0048_Y = PCSI0048.iloc[:,17:]  #Exclude CHROM_y, POS_y, END

In [72]:
print PCSI0048_X.shape
print PCSI0048_Y.shape

(125625, 13)
(125625, 134)


In [73]:
PCSI0048_X[:2]

,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias
0,47,9,1519,64,69,29,74,GCA,2.026144,0.716418,NaN,50859,0.281250
1,117,9,3971,139,119,0,48,CTT,2.748201,0.512438,downstream,47331,0.482014


In [74]:
PCSI0048_Y[:2]

,X2399010,X2642563,X2642565,X2673220,X2673230,X2701045,X2701049,X2811106,X2811111,X2811113,X2811116,X2811397,X2812914,X2813253,X2813265,X2813385,X2813460,X2813755,X2813758,X2813762,
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...


In [75]:
totalSample = PCSI0048_Y.shape[0]
PCSI0048_Y_Val = np.zeros((totalSample,1)) #Will be our label now

print totalSample
print PCSI0048_Y_Val.shape

125625
(125625, 1)


In [76]:
PCSI0048_Y.shape

(125625, 134)

In [77]:
PCSI0048_Y.values[0].shape

(134,)

In [78]:
for i in range(totalSample):
    PCSI0048_Y_Val[i] = np.argmax(np.bincount(PCSI0048_Y.values[i]))

In [79]:
PCSI0048_Y_Val_1 = pd.DataFrame(PCSI0048_Y_Val)
PCSI0048_Y_Val_1.columns = ['Label']

In [80]:
PCSI0048_Y_Val_1[:3]

,Label
0,0
1,0
2,0


In [81]:
PCSI0048_dataframe = pd.concat([PCSI0048_X, PCSI0048_Y_Val_1],axis=1)

In [82]:
PCSI0048_dataframe.shape

(125625, 14)

In [83]:
PCSI0048_dataframe[:2]

,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias,Label
0,47,9,1519,64,69,29,74,GCA,2.026144,0.716418,NaN,50859,0.281250,0
1,117,9,3971,139,119,0,48,CTT,2.748201,0.512438,downstream,47331,0.482014,0


In [84]:
PCSI0048_dataframe.to_csv('PCSI0048.csv')

In [85]:
#Now we have variable importance ...lets do analysis

In [31]:
%load_ext rpy2.ipython

In [32]:
%%R
# Read in the data
PCSI0048Frame = read.csv("PCSI0048.csv")
str(PCSI0048Frame)

'data.frame':	125625 obs. of  15 variables:
 $ X              : int  0 1 2 3 4 5 6 7 8 9 ...
 $ RefAllele      : int  47 117 124 122 116 172 97 18 30 63 ...
 $ NonRefAllele   : int  9 9 21 16 26 17 26 4 3 9 ...
 $ BaseQual       : int  1519 3971 4497 4151 4419 5884 3835 622 1079 1751 ...
 $ TumourCoverage : int  64 139 155 144 146 194 126 23 33 84 ...
 $ NormalCoverage : int  69 119 145 151 178 168 65 12 11 121 ...
 $ MapQual        : num  29 0 16 0 0 0 0 10 28 0 ...
 $ ReadPosition   : num  74 48 44 44 50 57 59 28 71 46 ...
 $ Trinucleotide  : Factor w/ 34 levels "ACA","ACC","ACG",..: 18 17 16 13 13 9 13 9 1 3 ...
 $ HomopolymerRate: num  2.03 2.75 2.1 2.41 2.61 ...
 $ GCcontent      : num  0.716 0.512 0.597 0.592 0.632 ...
 $ GenomicLocation: Factor w/ 16 levels "","downstream",..: 1 2 8 7 8 8 5 1 5 2 ...
 $ DistanceSNP    : int  50859 47331 45236 44449 43848 41251 11452 3360 3677 17695 ...
 $ StrandBias     : num  0.281 0.482 0.813 0.736 0.473 ...
 $ Label          : num  0 0 0 0 0 

In [33]:
%%R
#sum(is.na(CPCG0183Frame$StrandBias))
#sum(is.na(IS1Frame$ReadPosition)) 
sum(is.na(PCSI0048Frame)) 

[1] 0


In [34]:
#%%R
#CPCG0235Frame$ReadPosition[is.na(CPCG0235Frame$ReadPosition)] = mean(CPCG0235Frame$ReadPosition, na.rm=TRUE)
#CPCG0235Frame$StrandBias[is.na(CPCG0235Frame$StrandBias)] = mean(CPCG0235Frame$StrandBias, na.rm=TRUE)

#print (sum(is.na(CPCG0235Frame$ReadPosition)) )
#print (sum(is.na(CPCG0235Frame$StrandBias)) )

In [35]:
%%R
sum(is.na(PCSI0048Frame))  #No NA value now

[1] 0


In [36]:
%%R
write.csv(PCSI0048Frame, file = "PCSI0048_imputed.csv")  #Cleaned CSV without NA

In [37]:
%%R
library(h2o)
localH2O = h2o.init(ip = "localhost", port = 54321, startH2O = TRUE,min_mem_size = "3g")

/usr/local/lib/python2.7/dist-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: statmod

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/usr/local/lib/python2.7/dist-packages/rpy2/robjects/functions.py:106: UserWarning: 
----------------------------------------------------------------------

Your next step is to start H2O:
    > h2o.init()

For H2O package documentation, ask for help:
    > ??h2o

After starting H2O, you can use the Web UI at http://localhost:54321
For more information visit http://docs.h2o.ai

----------------------------------------------------------------------


  res = super(Function, self).__call__(*new_args, **new_kwargs)
/usr/local/lib/python2.7/dist-packages/rpy2/robjects/functions.py:106: UserWarning: 
Attaching package: ‘h2o’


  res = super(Function, self).__call__(*new_args, **new_kwargs)
/usr/local/lib/python2.7/dist-packages/rpy2/robjects/functions.py:106: UserWarning: The following objects are masked from ‘

 Connection successful!

R is connected to the H2O cluster: 
    H2O cluster uptime:         1 hours 24 minutes 
    H2O cluster version:        3.9.1.3435 
    H2O cluster name:           H2O_started_from_R_rbhat_xqb196 
    H2O cluster total nodes:    1 
    H2O cluster total memory:   26.56 GB 
    H2O cluster total cores:    56 
    H2O cluster allowed cores:  2 
    H2O cluster healthy:        TRUE 
    H2O Connection ip:          localhost 
    H2O Connection port:        54321 
    H2O Connection proxy:       NA 
    R Version:                  R version 3.3.0 beta (2016-03-30 r70404) 



In [38]:
%%R
## Import CSV as H2O:
csvPath = "/home/rbhat/Workspace/S3Lab_Projects/SMC-MetaChallange/PCSI0048_imputed.csv"
PCSI0048.hex          = h2o.importFile(path = csvPath, destination_frame = "PCSI0048.hex") 
train                 = as.data.frame(PCSI0048.hex)

str(train)

  |======================================================================| 100%
'data.frame':	125625 obs. of  16 variables:
 $ C1             : int  1 2 3 4 5 6 7 8 9 10 ...
 $ X              : int  0 1 2 3 4 5 6 7 8 9 ...
 $ RefAllele      : int  47 117 124 122 116 172 97 18 30 63 ...
 $ NonRefAllele   : int  9 9 21 16 26 17 26 4 3 9 ...
 $ BaseQual       : int  1519 3971 4497 4151 4419 5884 3835 622 1079 1751 ...
 $ TumourCoverage : int  64 139 155 144 146 194 126 23 33 84 ...
 $ NormalCoverage : int  69 119 145 151 178 168 65 12 11 121 ...
 $ MapQual        : num  29 0 16 0 0 0 0 10 28 0 ...
 $ ReadPosition   : num  74 48 44 44 50 57 59 28 71 46 ...
 $ Trinucleotide  : Factor w/ 34 levels "ACA","ACC","ACG",..: 18 17 16 13 13 9 13 9 1 3 ...
 $ HomopolymerRate: num  2.03 2.75 2.1 2.41 2.61 ...
 $ GCcontent      : num  0.716 0.512 0.597 0.592 0.632 ...
 $ GenomicLocation: Factor w/ 15 levels "downstream","exonic",..: NA 1 7 6 7 7 4 NA 4 1 ...
 $ DistanceSNP    : int  50859 47331 45236 

In [39]:
%%R

train$Trinucleotide   = as.factor(train$Trinucleotide)
train$GenomicLocation = as.factor(train$GenomicLocation)
train$Label           = as.factor(train$Label)

train_h2o             = as.h2o(train)

  |======================================================================| 100%


In [40]:
%%R
#Training a deep learning model:---------------------------------------------------------#
model <- h2o.deeplearning( x = 3:15, # column numbers for predictors
                           y = 16,            # column number for label
                           training_frame = train_h2o,
                           activation = "RectifierWithDropout", 
                           input_dropout_ratio = 0.2,
                           hidden_dropout_ratios = c(0.5,0.5),
                           balance_classes = TRUE, 
                           hidden = c(10,8),  ## two hidden layers
                           epochs = 50,
                           variable_importances = T)

  |======================================================================| 100%


In [41]:
%%R
var_importance = h2o.varimp(model)
#print(var_importance)
h2o.varimp(model)

Variable Importances: 
        variable relative_importance scaled_importance percentage
1   NonRefAllele            1.000000          1.000000   0.238021
2 NormalCoverage            0.357612          0.357612   0.085119
3       BaseQual            0.353273          0.353273   0.084086
4 TumourCoverage            0.270955          0.270955   0.064493
5      RefAllele            0.243528          0.243528   0.057965

---
                    variable relative_importance scaled_importance percentage
57         Trinucleotide.GTA            0.009779          0.009779   0.002328
58         Trinucleotide.GCA            0.009765          0.009765   0.002324
59         Trinucleotide.TTT            0.009264          0.009264   0.002205
60                StrandBias            0.005648          0.005648   0.001344
61              ReadPosition            0.004933          0.004933   0.001174
62 Trinucleotide.missing(NA)            0.000000          0.000000   0.000000


In [42]:
%%R
write.table(var_importance, file="PCSI0048_variable_importance.csv")

In [43]:
var_importance = pd.read_csv(open('PCSI0048_variable_importance.csv','r'),sep=' ')
#pd.read_csv('CPCG0100_variable_importance.csv')

In [44]:
var_importance[:5]

,variable,relative_importance,scaled_importance,percentage
1,NonRefAllele,1.000000,1.000000,0.238021
2,NormalCoverage,0.357612,0.357612,0.085119
3,BaseQual,0.353273,0.353273,0.084086
4,TumourCoverage,0.270955,0.270955,0.064493
5,RefAllele,0.243528,0.243528,0.057965


In [45]:
GenomicFeatures_PCSI0048[:5]

,CHROM,POS,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias
0,chr1,10583,47,9,1519,64,69,29,74,GCA,2.026144,0.716418,NaN,50859,0.281250
1,chr1,14111,117,9,3971,139,119,0,48,CTT,2.748201,0.512438,downstream,47331,0.482014
2,chr1,16206,124,21,4497,155,145,16,44,CTG,2.101911,0.597015,ncRNA_intronic,45236,0.812903
3,chr1,16993,122,16,4151,144,151,0,44,CCT,2.406897,0.592040,ncRNA_exonic,44449,0.736111
4,chr1,17594,116,26,4419,146,178,0,50,CCT,2.605634,0.631841,ncRNA_intronic,43848,0.472603


In [46]:
var_imp_dict = {}

for i in range(var_importance.shape[0]):
    var_imp_dict[var_importance.iloc[i,0]] = var_importance.iloc[i,1]

var_imp_dict

{'BaseQual': 0.35327306389808705,
 'DistanceSNP': 0.11721476167440402,
 'GCcontent': 0.012889640405774099,
 'GenomicLocation.UTR3': 0.0271538197994232,
 'GenomicLocation.UTR5': 0.050726816058158902,
 'GenomicLocation.downstream': 0.030489305034279799,
 'GenomicLocation.exonic': 0.0588451735675335,
 'GenomicLocation.exonic;splicing': 0.0374986454844475,
 'GenomicLocation.intergenic': 0.0108668636530638,
 'GenomicLocation.intronic': 0.0099646290764212591,
 'GenomicLocation.missing(NA)': 0.020019568502903002,
 'GenomicLocation.ncRNA_UTR3': 0.053101282566785799,
 'GenomicLocation.ncRNA_UTR5': 0.046021103858947802,
 'GenomicLocation.ncRNA_exonic': 0.039995804429054302,
 'GenomicLocation.ncRNA_intronic': 0.022748973220586801,
 'GenomicLocation.ncRNA_splicing': 0.050066940486431101,
 'GenomicLocation.splicing': 0.046210128813982003,
 'GenomicLocation.upstream': 0.026415338739752801,
 'GenomicLocation.upstream;downstream': 0.0347137115895748,
 'HomopolymerRate': 0.162501156330109,
 'MapQual': 

In [47]:
GenomicFeatures_PCSI0048.columns

Index([u'CHROM', u'POS', u'RefAllele', u'NonRefAllele', u'BaseQual', u'TumourCoverage', u'NormalCoverage', u'MapQual', u'ReadPosition', u'Trinucleotide', u'HomopolymerRate', u'GCcontent', u'GenomicLocation', u'DistanceSNP', u'StrandBias'], dtype='object')

In [48]:
GenomicFeatures_PCSI0048.dtypes

CHROM               object
POS                  int64
RefAllele            int64
NonRefAllele         int64
BaseQual             int64
TumourCoverage       int64
NormalCoverage       int64
MapQual            float64
ReadPosition       float64
Trinucleotide       object
HomopolymerRate    float64
GCcontent          float64
GenomicLocation     object
DistanceSNP          int64
StrandBias         float64
dtype: object

In [49]:
GenomicFeatures_PCSI0048.iloc[0,13]

50859

In [50]:
GenomicFeatures_PCSI0048[:2]
#print GenomicFeatures_IS1.iloc[0,2]

,CHROM,POS,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias
0,chr1,10583,47,9,1519,64,69,29,74,GCA,2.026144,0.716418,NaN,50859,0.281250
1,chr1,14111,117,9,3971,139,119,0,48,CTT,2.748201,0.512438,downstream,47331,0.482014


In [51]:
noOfSample = GenomicFeatures_PCSI0048.shape[0]

PCSI0048_Dict = {}

for i in range(noOfSample):
    
    pos = GenomicFeatures_PCSI0048.iloc[i,1] #gets the pos value
    
    RefAllele      = GenomicFeatures_PCSI0048.iloc[i,2]*var_imp_dict['RefAllele']
    NonRefAllele   = GenomicFeatures_PCSI0048.iloc[i,3]*var_imp_dict['NonRefAllele']
    BaseQual       = GenomicFeatures_PCSI0048.iloc[i,4]*var_imp_dict['BaseQual']
    TumourCoverage = GenomicFeatures_PCSI0048.iloc[i,5]*var_imp_dict['TumourCoverage']
    NormalCoverage = GenomicFeatures_PCSI0048.iloc[i,6]*var_imp_dict['NormalCoverage']
    MapQual        = GenomicFeatures_PCSI0048.iloc[i,7]*var_imp_dict['MapQual']
    ReadPosition   = GenomicFeatures_PCSI0048.iloc[i,8]*var_imp_dict['ReadPosition']
    
    Trinucleotide  = 0
    if isinstance(GenomicFeatures_PCSI0048.iloc[i,9], str):
        Trinucleotide  = var_imp_dict['Trinucleotide.'+str(GenomicFeatures_PCSI0048.iloc[i,9])]
    HomopolymerRate= GenomicFeatures_PCSI0048.iloc[i,10]*var_imp_dict['HomopolymerRate']
    GCcontent      = GenomicFeatures_PCSI0048.iloc[i,11]*var_imp_dict['GCcontent']
    
    GenomicLocation = 0
    if isinstance(GenomicFeatures_PCSI0048.iloc[i,12], str):
        GenomicLocation= var_imp_dict['GenomicLocation.'+str(GenomicFeatures_PCSI0048.iloc[i,12])]
    DistanceSNP    = GenomicFeatures_PCSI0048.iloc[i,13]*var_imp_dict['DistanceSNP']
    StrandBias     = GenomicFeatures_PCSI0048.iloc[i,14]*var_imp_dict['StrandBias']
    
    #print 'RefAllele=',RefAllele,' NonRefAllele=',NonRefAllele,' BaseQual=',BaseQual,' TumourCoverage=',TumourCoverage
    #print 'NormalCoverage=',NormalCoverage,' MapQual=',MapQual,' ReadPosition=',ReadPosition,' Trinucleotide=',Trinucleotide
    #print 'HomopolymerRate=',HomopolymerRate,' GCcontent=',GCcontent,' GenomicLocation=',GenomicLocation,' Distance=',Distance
    #print 'StrandBias=',StrandBias
    PCSI0048_Dict[pos] = RefAllele+NonRefAllele+BaseQual+TumourCoverage+NormalCoverage+MapQual+ReadPosition+Trinucleotide+ \
                         HomopolymerRate+GCcontent+GenomicLocation+DistanceSNP+StrandBias

In [52]:
#Sorting dictionary
PCSI0048_Dict1 = [(k,v) for v,k in sorted([(v,k) for k,v in PCSI0048_Dict.items()],reverse=True)]

In [53]:
PCSI0048_Dict_Frame = pd.DataFrame(PCSI0048_Dict1, columns=['POS', 'Value'])

In [54]:
PCSI0048_Dict_Frame.to_csv("PCSI0048_Dict_Frame.csv")

In [86]:
PCSI0048_Dict_Frame = pd.read_csv("PCSI0048_Dict_Frame.csv")

In [87]:
PCSI0048_Dict_Frame[:10]

,Unnamed: 0,POS,Value
0,0,58883954,3590576.225037
1,1,58854810,3582710.577241
2,2,59324554,3579632.818998
3,3,58996968,3565857.430643
4,4,58996552,3552562.826076
5,5,58982222,3550494.169402
6,6,58993986,3549986.716409
7,7,58979561,3549566.595554
8,8,58997341,3548875.656092
9,9,58997320,3548816.431994


In [88]:
GenomicFeatures_PCSI0048_1 = pd.concat([PCSI0048.iloc[:,0:16], PCSI0048_Y_Val_1],axis=1)

In [89]:
rows = PCSI0048_Dict_Frame['POS'][:20].values
print rows
GenomicFeatures_PCSI0048_Top20 = GenomicFeatures_PCSI0048_1.loc[GenomicFeatures_PCSI0048_1['POS_x'].isin(rows)]

[58883954 58854810 59324554 58996968 58996552 58982222 58993986 58979561
 58997341 58997320 58988536 59015360 58978645 59032171 58983185 58893768
 59021207 59029604 59032176 58978359]


In [91]:
GenomicFeatures_PCSI0048_Top20[:10]

,CHROM,POS_x,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias,POS_y,Label
125449,chrY,58854810,4440,565,149153,5023,7881,0,47,ATT,2.874074,0.378109,NaN,30066115,0.099741,58854809,0
125461,chrY,58883954,4526,661,160712,5209,8437,0,47,ATC,2.206897,0.368159,NaN,30095259,0.236514,58883953,0
125467,chrY,58893768,1438,213,49342,1666,3013,0,48,GTA,2.604478,0.353234,NaN,30105073,0.210084,58893767,0
125514,chrY,58978359,780,76,24224,861,995,29,35,TCG,2.080000,0.442786,NaN,30189664,0.572590,58978358,0
125515,chrY,58978645,928,17,24895,955,1097,29,73,GTG,2.401408,0.492537,NaN,30189950,0.468063,58978644,0
125518,chrY,58979561,846,24,28075,870,1016,29,56,TCA,2.200000,0.442786,intergenic,30190866,0.477011,58979560,0
125532,chrY,58982222,842,114,29463,959,1050,29,61,GCA,2.270833,0.482587,NaN,30193527,0.453597,58982221,0
125534,chrY,58983185,659,127,23560,804,853,29,42,ACT,2.052632,0.487562,NaN,30194490,0.366915,58983184,0
125540,chrY,58988536,591,127,22543,811,961,57,41,ACA,1.690476,0.502488,NaN,30199841,0.564735,58988535,0
125546,chrY,58993986,547,222,24111,870,1029,29,38,TTA,1.769697,0.487562,NaN,30205291,0.387356,58993985,0


In [58]:
GenomicFeatures_PCSI0048_Top20.to_csv('GenomicFeatures_PCSI0048_Top20.csv')